In [6]:
import pyodbc
import pandas as pd
import numpy as np
import time
import logging

from dupdetectorML import DatasetBuilder, Trainer, SaveModel

Configure parameters 

In [7]:
DATASOURCE_COLUMNS = ['FindCode', 'Name', 'Phone', 'Fax', 'OtherPhone', \
                                  'Email', 'WebsiteURL', 'MailingAddressFreeform', \
                                  'MailingAddressCity', 'MailingAddressPostalCode', 'MailingAddressState']
DATASOURCE_INDEX = 'FindCode'
ONE_HOT_ENCONDING_COLUMNS = {'MailingAddressState': ['nunavut ',
                                                        'saskatchewan',
                                                        'ontario  ',
                                                        'alberta',
                                                        'british columbia ',
                                                        'prince edward island ',
                                                        'yukon territory',
                                                        'newfoundland',
                                                        'northwest territories',
                                                        'new brunswick',
                                                        'manitoba',
                                                        'nova scotia ',
                                                        'quebec']}
TEXT_METRICS = ['ratio','partial_ratio','token_sort_ratio','token_set_ratio','distance',
                'l_ratio','jaro','jaro_winkler','setratio','seqratio','longestnumericseq']
PASS_THROUGH_COMULNS = ['FindCode','MailingAddressState']
HIGH_IMPORTANCE_COLUMNS = ['Name']

alteration_rules = [
    {
        'rule_Replace': ['none',''],
        'rule_RandomTypo': ['alpha', 2, 'replace'],
        'rule_ScrambleWords': [],
        'rule_DuplicateNumericSequence': [2],
        'rule_RemoveSpecialSymbols': [],
        'rule_RemoveStopWords': [],
        'rule_IncreaseWeightOfShortWords':[]
    }
]

COLUMN_ALTERATIION_RULES = []
for i in range(0, len(alteration_rules)):
    COLUMN_ALTERATIION_RULES.append(
        {
            1: alteration_rules[i],
            2: alteration_rules[i],                    
            3: alteration_rules[i],
            4: alteration_rules[i],
            5: alteration_rules[i],
            6: alteration_rules[i],
            7: alteration_rules[i],
            8: alteration_rules[i],
            9: alteration_rules[i]
        })

Fetch data from source. Meta should match configuration above

In [8]:
df_source = pd.read_csv('train_input.csv', header=0, names=DATASOURCE_COLUMNS)
df_source.FindCode = df_source.FindCode.astype(str).str.lower()
df_source.Name = df_source.Name.astype(str).str.lower()
df_source.Phone = df_source.Phone.astype(str).str.lower()
df_source.Fax = df_source.Fax.astype(str).str.lower()
df_source.OtherPhone = df_source.OtherPhone.astype(str).str.lower()
df_source.Email = df_source.Email.astype(str).str.lower()
df_source.WebsiteURL = df_source.WebsiteURL.astype(str).str.lower()
df_source.MailingAddressFreeform = df_source.MailingAddressFreeform.astype(str).str.lower()
df_source.MailingAddressCity = df_source.MailingAddressCity.astype(str).str.lower()
df_source.MailingAddressPostalCode = df_source.MailingAddressPostalCode.astype(str).str.lower()
df_source.MailingAddressState = df_source.MailingAddressState.astype(str).str.lower()

Create training dataset

In [ ]:
builder = DatasetBuilder(DATASOURCE_COLUMNS, DATASOURCE_INDEX, 
                         ONE_HOT_ENCONDING_COLUMNS, TEXT_METRICS, 
                         PASS_THROUGH_COMULNS, COLUMN_ALTERATIION_RULES,
                         HIGH_IMPORTANCE_COLUMNS, 8, logging.DEBUG)
training_df = builder.generateTrainingDataset(df_source)

2020-04-18 14:28:01,532 - root - INFO - Column: MailingAddressCity_x, ratio metric took: 99.39214253425598 seconds
2020-04-18 14:29:43,045 - root - INFO - Column: MailingAddressCity_x, partial_ratio metric took: 101.51232695579529 seconds
2020-04-18 14:31:27,010 - root - INFO - Column: MailingAddressCity_x, token_sort_ratio metric took: 103.96429371833801 seconds
2020-04-18 14:33:10,301 - root - INFO - Column: MailingAddressCity_x, token_set_ratio metric took: 103.28965711593628 seconds
2020-04-18 14:34:58,127 - root - INFO - Column: MailingAddressCity_x, distance metric took: 107.82645058631897 seconds
2020-04-18 14:36:43,892 - root - INFO - Column: MailingAddressCity_x, l_ratio metric took: 105.76379656791687 seconds
2020-04-18 14:38:29,552 - root - INFO - Column: MailingAddressCity_x, jaro metric took: 105.65849876403809 seconds
2020-04-18 14:40:16,771 - root - INFO - Column: MailingAddressCity_x, jaro_winkler metric took: 107.21825337409973 seconds


In [ ]:
trainer = Trainer(training_df, 0.2)
SaveModel(trainer.execute(), "TrainedModel.sav")